In [1]:
import pandas as pd
import numpy as np
from ahp import get_priorities, get_relative_consistency_index, recreate_consitant_matrix
from pathlib import Path

In [6]:
hierarchy = ["comparisons_type.csv", "comparisons_alternatives.csv"]

datadir = "../dataset/ahp/"

for file in hierarchy:
    data = pd.read_csv(Path(datadir).joinpath(file))

    for parent in data.parent.unique()[::-1]:
        comparisons = pd.DataFrame(data.loc[data["parent"] == parent, :])
        
        unique_elements = list(set(comparisons["Element1"].unique().tolist() + comparisons["Element2"].unique().tolist()))

        result = pd.DataFrame(index=unique_elements, columns=unique_elements)

        for element in unique_elements:
            result.at[element, element] = 1.0
            
        for comparison_id in comparisons.index:
            el1 = comparisons.at[comparison_id, "Element1"]
            el2 = comparisons.at[comparison_id, "Element2"]
            judgement = comparisons.at[comparison_id, "Judgement"]
            result.at[el1, el2] = float(judgement)
            result.at[el2, el1]  = 1 / float(judgement)


        comparison_matrix_raw = result.to_numpy(dtype=float)
        assert not np.isnan(comparison_matrix_raw).any()

        priorities = get_priorities(comparison_matrix_raw)

        relative_consistency_index = get_relative_consistency_index(comparison_matrix_raw)

        print(f"-------------------------")
        print(f"For pairwise comparison of: {result.index} in respect to {parent}")
        print(f"\tOriginal comparison matrix:")
        print(comparison_matrix_raw)
        print(f"\tRelative Consistency Index = {relative_consistency_index}")
        print(f"\tRecreated Consistent Matrix")
        print(recreate_consitant_matrix(priorities))
        print()


-------------------------
For pairwise comparison of: Index(['EnginePower', 'SailArea', 'Width'], dtype='object') in respect to Technical
	Original comparison matrix:
[[1.         0.25       6.        ]
 [4.         1.         9.        ]
 [0.16666667 0.11111111 1.        ]]
	Relative Consistency Index = 0.09297183953014879
	Recreated Consistent Matrix
[[ 1.          0.34668064  4.32674871]
 [ 2.88449914  1.         12.48050294]
 [ 0.23112042  0.08012498  1.        ]]

-------------------------
For pairwise comparison of: Index(['Deposit', 'PricePerDay'], dtype='object') in respect to Money
	Original comparison matrix:
[[1.    0.125]
 [8.    1.   ]]
	Relative Consistency Index = 0.0
	Recreated Consistent Matrix
[[1.    0.125]
 [8.    1.   ]]

-------------------------
For pairwise comparison of: Int64Index([270, 274, 307, 280, 249, 252, 286, 255], dtype='int64') in respect to Width
	Original comparison matrix:
[[1.         1.         0.2        1.         4.         4.
  0.33333333 1. 